<p style='text-align: center'><a href=https://www.biozentrum.uni-wuerzburg.de/cctb/research/supramolecular-and-cellular-simulations/>Supramolecular and particleular Simulations</a> (Prof. Fischer)<br>Center for Computational and Theoretical Biology - CCTB<br>Faculty of Biology, University of Würzburg</p>

<p style='text-align: center'><br><br>We are looking forward to your comments and suggestions. Please send them to: <br><br></p>
    
 <p style='text-align: center'>   <a href=andreas.kuhn@uni.wuerzburg.de>andreas.kuhn@uni.wuerzburg.de</a> or <a href=sabine.fischer@uni.wuerzburg.de>sabine.fischer@uni.wuerzburg.de</a></p>

<h1><p style='text-align: center'> Introduction to Julia </p></h1>


## Example Project

### Introduction 

Until now, you have learned to use different data types, to write your own functions, to import/export/analyse data and to create beautiful plots. The question that you might be asking yourself is: "How does all of this come together?"

A hands-on exercise will provide an example. You will write a simple agent-based simulation. This involves developing a small model with randomly moving agents on a grid, using all the stuff you have already learned.




### Structure

The simulation consists of agents occupying an underlying grid. Each grid cell can only be occupied by one agent and can be accessed by its x and y coordinates. In the graphic below, a 6*7 slice of a potentially much bigger grid is shown. 


<div>
<img src="Grid_visu_1.png"width="800" />
</div>

#### 1. Import used packages 
 
The first step in creating such a system is (as always :) ) to import the needed packages.
 
Note: Theoretically, you could import these at any point between the beginning and the point in the program where you actually use them. However, for clarity/readability, you should always do this at the beginning of your notebook/script.

In [ ]:
using Pkg
Pkg.add("GLMakie")

In [ ]:
using CSV, DataFrames
using GLMakie,ColorSchemes, Random

In this case, the packages `CSV` and `DataFrames` are needed for transforming and saving the output data as dataframes. Additionally, the package `GLMakie` which is the gpu-power backend for `Makie` has to be installed and used. It works exactly the same way as `CairoMakie`, but outputs its plots in a separate interactive window. Therefore, it is better suited for animations or interactive plots.  
The package `ColorSchemes` provides additional colormaps for `Makie`. One function from the package `random` of the Julia standard library is also used. 

 #### 2. Define Parameters
The second step is to set all defining parameters of the simulation. In this case, these are the size of the grid, the number of agents, the time how long the simulation should run and the configuration/geometry in which the agents should be placed on the grid.



In [ ]:
gridsize = 100                                
particle_number = 200                          
timesteps = 300                                
#starting_config = "square"
starting_config = "random"

Note: As the background of one of the authors is physics, we arbitrarily  declare the agents to be representing particles. But an agent can represent almost anything, especially when the only implemented rule is a random walk which is very similar for a lot of systems (atoms on lattice, molecules in a solution, stars in galaxys, insects on the ground, humans in a pedestrian zone,... ).     

 #### 3.  Initialize system
The next step is to create the objects of the simulation. Ideally, these should only depend on the defined parameters. Here, the grid and the particles/agents have to be created. 

##### 3.1 Structure of data

The empty square grid can be created with the parameter gridsize and the `zeros()` function. A zero at a given grid point means that it is unoccupied. 

In [ ]:
grid_test = zeros(Int64, gridsize, gridsize)

The system has to be populated with particles. In this case, a twofold approach is used. The positions of the particles is saved implicitly in the grid and explicitly in an additional vector. You will later see why.

To avoid slow `Any` arrays, it is best to explicitly define the type of every array used in the simulation. There are two different ways to initialize an array with a specific type. You have already seen the `type[]` constructor in the Datatype notebook. But there exists an even more powerful syntax to define arrays:

- `Array{type}(undef,dims)` where dims gives the length of the array in each dimension, e.g. dims = 10 creates a 10 elements long 1D array, dims = 5,5,5 creates a 5x5x5 3D-array (sometimes called a tensor) 

 The big advantage of this approach is that it can be nested within itself to create arbitrarily complex data structures.

Just to give you a small example how the constructor works: 

In [ ]:
hans = Array{Int64}(undef,0)
dieter = Int64[]
#test if hans and dieter represent the same data structure
println(hans == dieter)

Note: The `undef` keyword says that the content is not yet defined. In the case of an array with length zero, this does not matter. But for a length > 0, the values in the created array will assume the unchanged bytes in the assigned memory location. For arrays consisting of primitve dataytpes like `Int64` or `Float64`, the values will be random. More complex datatypes (`dicts, arrays,...)` need a certain structure in the underlying memory. That means that the bytes can, but not necessarily need to, be a valid representation of said datatype and might be completely unusable (and cause errors) until they get overwritten.   

As the single particles are solely represented by their coordinates, a suitable data structure is a vector with `length = particle_number` containing (sub)-vectors in the format `[x,y]`. Such a vector can be created as follows:

In [ ]:
# vector of Int64-vectors with size 0
particle_list_test = Vector{Vector{Int64}}(undef,0)

Sub vectors can be added to the particle_list with the `push!()` function.  

In [ ]:
push!(particle_list_test,[2,3])

In [ ]:
push!(particle_list_test,[65,12])

Let's remove the 2 example positions to start with the real positions: 

In [ ]:
pop!(particle_list_test)
pop!(particle_list_test)
particle_list_test

Note: This approach is the most performant, but also the most complicated way of setting up your data. If speed is not a concern, you can also make use of the `[]` constructor, which creates an `Any` array. Such arrays can do everything (and much more ;) ) as the one we are using, with the price of being slower. But as Julia always claims to be a fast language, we should not intentionally handicap in that regard. 

##### 3.2 Create particles 

When the particles are created, they have to be stored in the grid and the particle_list. In the case of a random config, the values of `x` and `y` are assigned randomly. But as two particles cannot occupy the same grid point, it is necessary to check if the assigned grid point is already occupied.

In [ ]:
if starting_config == "random"
    i = 1
    while i <= particle_number
        x = rand(1:gridsize)
        y = rand(1:gridsize)
        # check if grid point is already occupied and only if it is empty 
        # create a particle and increase the counter by one 
        if grid_test[x,y] == 0 
            grid_test[x,y] = i
            push!(particle_list_test,[x,y])
            i += 1
        end
    end
end

If the starting config is a square, a bit more work has to be done. To simplify things, the particle number is rounded to the next square number. After that, the square is filled column by column with two for-loops.   

In [ ]:
if starting_config == "square"
    #calculate edge length of square of particles
    edge_length = round(Int64,sqrt(particle_number))
    # round particle_number to closest square number
    particle_number = edge_length^2
    #calculate starting x and y value of corner of square on grid
    x_corner = round(Int64,gridsize/2-edge_length/2)
    y_corner = round(Int64,gridsize/2-edge_length/2)
    #all the rounding is needed to get to discrete grid points
    i = 1
    for x = 1:edge_length
        for y = 1:edge_length
            grid_test[x_corner+x, y_corner+y] = i
            push!(particle_list_test,[x_corner+x,y_corner+y])
            i += 1
        end
    end
end

In order to make things clearer, it is good practice to encapsulate different parts of your program into functions. In this case, two functions are used. One function to create and return the grid and particle_list and one to populate them according to the given starting configuration: 

In [ ]:
function create_objects(Gridsize, Particle_number)
    Grid = zeros(Int64, Gridsize, Gridsize)
    Particle_list = Vector{Vector{Int64}}(undef,0)
    return Grid, Particle_list
end

function populate_sys!(Grid, Particle_list, Starting_config,Gridsize, Particle_number) 
    if Starting_config == "random"
        i = 1
        while i <= Particle_number
            x = rand(1:Gridsize)
            y = rand(1:Gridsize)
            if Grid[x,y] == 0 
                Grid[x,y] = i
                push!(Particle_list,[x,y])
                i += 1
            end
        end
    end
    if Starting_config == "square"
        #calculate edge length of square of particles
        Edge_length = round(Int64,sqrt(Particle_number))
        # round particle_number to closest square number
        Particle_number = Edge_length^2
        #calculate starting x and y value of corner of square on grid
        X_corner = round(Int64,Gridsize/2-Edge_length/2)
        Y_corner = round(Int64,Gridsize/2-Edge_length/2)
        #all the rounding is needed to get to discrete grid points
        i = 1
        for x = 1:Edge_length
            for y = 1:Edge_length
                Grid[X_corner+x, Y_corner+y] = i
                push!(Particle_list,[X_corner+x,Y_corner+y])
                i += 1
            end
        end
    end
    #return everything that could have changed
    return Grid, Particle_list, Particle_number
end

To make it clear that the local variables inside the functions are different from the ones previously used in the global scope, they start with capital letters (but would not need to ;) ). 

 #### 4.  Plot starting config
 Now let's execute the functions and plot the starting config to see if everything is working as intended.

In [ ]:
grid, particle_list = create_objects(gridsize, particle_number)
grid, particle_list, particle_number = populate_sys!(grid, particle_list, "square",gridsize, particle_number);

This time, the plotting part is directly encapsulated into a function. You will later see why. 

In [ ]:
function plot_sim(particle_list, Gridsize)
    # make plot look nicer in black ; )
    set_theme!(theme_black())

    # using array comprehensions to create positions array of x and y
    x = [i[1] for i in particle_list]
    y = [i[2] for i in particle_list]
    # calculate the distance r for every particle from the center
    
    ## r is an array that holds the distance r from every particle to the center of the grid
    # it is used for plotting, so that the particles have different colors 
    # depending on their distance from the center 
    r = (((x.-Gridsize/2).^2+(y.-Gridsize/2).^2).^(1/2))

    Fig1 = Figure(resolution = (1000,1000))
    Ax1 =Axis(Fig1[1,1],title = "Current State",titlesize = 35)
    xlims!(Ax1,0,Gridsize)
    ylims!(Ax1,0,Gridsize)
    # using the distance r together with a colormap to give the particles different colors
    Scatty = scatter!(Ax1,x,y,color = r,colormap = :dense,label = "particle",marker = :circle,markersize = 8)

    return Fig1, Ax1 , Scatty
end

In [ ]:
fig1, ax1, scatty = plot_sim(particle_list,gridsize)
display(fig1)

A separate Makie window should open, where the plot is displayed. 

Hint: It can be annoying that the Makie window does not automatically appear on top of the other windows when creating a new plot. On Windows, the tool [pinwin](https://sourceforge.net/projects/pinwin/) can be used to permanently put Makie (or any other window) on the top of all used windows.

#### 5. Update function
The next step in a simulation is to write an update function. This function evolves the simulation from its starting configuration through time in discrete intervals. 

It defines the basic rules/interactions of the simulatuion as well as the time discretization.

<div>
<img src="Grid_visu_2.png"width="800" /> 
</div>
    
In each time step, every particle randomly picks one of the eight possible movement vectors to a neighbouring grid site.   
</div>
<img src="Grid_visu_3.png"width="800" />
</div>


If the selected neighbouring grid site is empty, the particle moves to that grid site and its position gets updated in the grid and the particle list.

</div>
<img src="Grid_visu_4.png"width="800" />
</div>

This process is repeated for every particle in every timestep: 

In [ ]:
function update_sys(Grid, particle_list, Gridsize,Timesteps)                  # input arguments, are the starting configuration (Grid & particle_list) and the Gridsize and number of Timesteps
    particle_list_alltime =  Vector{Vector{Vector{Int64}}}(undef,0)           # super particle list which contains one particle_list per time step 
    mov_vec = [[0,1],[0,-1],[1,0],[-1,0],[1,1],[-1,-1],[-1,1],[1,-1]]     # possible movement vectors to the next grid point
    sequence_vec = collect(1:8)                                           # sequence_vec is used to access the mov_vec
    
    
    particle_list_copy = deepcopy(particle_list)                                  # make real copy (no reference) of particle_list and push it into particle_list_alltime 
    push!(particle_list_alltime,particle_list_copy)
    for t = 1:Timesteps                                                   # iterate over all timesteps
        for (j,particle) in  enumerate(particle_list)                             # iterate over all particles
            shuffle!(sequence_vec)                                        # use shuffle function from random to ensure random movement of particles
            for sque in sequence_vec
                x_next = particle[1] + mov_vec[sque][1]
                y_next = particle[2] + mov_vec[sque][2]
                
                if x_next >= 1 && x_next <= Gridsize && y_next >= 1 && y_next <= Gridsize    # check if grid point is out of bounds of grid, if so do nothing
                    if Grid[x_next,y_next] == 0 && Grid[particle[1],particle[2]] != 0                # check if target grid point is empty and sanity check if particle exists on previous grid point
                        Grid[particle[1],particle[2]] = 0 
                        Grid[x_next,y_next]  = j 
                        
                        particle[1] = x_next
                        particle[2] = y_next
                        break
                    end
                end                                                                          
            end                                                                              # if no grid point fullfils the condition, the particle does not move
        end
        particle_list_copy = deepcopy(particle_list)                                   # creating copy of particle list in order to avoid a pass by reference (see below for an explanation of this term). 
        push!(particle_list_alltime,particle_list_copy)
    end
    return(particle_list_alltime)
end

If you had a hard time following the function above, here is a short summary: 

Firstly, it creates an array called `particle_list_alltime` which is used to save a copy of the particle list in every time step.

Then, three nested loops are executed. The first one runs over all time steps. 

The second one runs over all particles (once per time step). 

The third one goes over all neighbouring grid points for every particle in random order with the help of the `mov_vec[]`, `sequence_vec[]` and the `shuffle!()` function. If a neighbouring grid point is empty, a particle moves there, the grid and particle_list get updated and the innermost loop breaks. If there is no empty grid point, nothing happens. 

#### Short interlude - Pass by value or by reference 

You may also wonder why the `deepcopy()` function is used before the particle_list is `push!()` into the `particle_list_alltime`. This is due to a concept that has not been introduced yet: [passing by reference](https://stackoverflow.com/questions/373419/whats-the-difference-between-passing-by-reference-vs-passing-by-value) (if you are interested in computer science click on the link). Essentially, it means that in `Julia` all mutable datatype (like arrays, dicts,...) are not copied when they are assigned to a new variable. The new variable just points to the same already existing object in memory. See small example below:   

In [ ]:
dieter = [2,5,34]
dieter2 = dieter
dieter2[2] = 7777
#even though we haven't changed dieter, it got modified as dieter2 is only a reference pointing to the same object
println(dieter, dieter === dieter2) 
# the === operator checks if two variables are refering to the same object

In [ ]:

dieter3 = [2,5,34]
dieter4 = deepcopy(dieter3)
dieter4[2] = 7777
#now dieter4 is a new object 
println(dieter3, dieter3 === dieter4)

#### Interlude end

Let's execute the `update_sys` function and plot the particle positions in the last time step. 

In [ ]:
particle_all = update_sys(grid,particle_list,gridsize,timesteps)

fig2, ax2, scatty2 = plot_sim(particle_all[end],gridsize)
display(fig2)

The particles have moved! For a more complex analysis, it's useful to convert the `particle_all` array into a data frame.  

You might be wondering why the simulation does not run directly on `DataFrames` instead of arrays. It is possible to do that and in many cases it would not make a big difference, but `DataFrames` are slower and use more memory. Therefore, if a simulation needs to be scalable to big particle numbers and/or time steps, arrays are a better choice. 

In [ ]:
function convert_to_DF(particle_list_alltime)
    particle_number = length(particle_list_alltime[1])
    Data = DataFrame(timestep = ones(Int64,particle_number),id= collect(1:particle_number),
        x = [i[1] for i in particle_list_alltime[1]],y = [i[2] for i in particle_list_alltime[1]])
    for j in 2:length(particle_list_alltime)
        append!(Data,DataFrame(timestep = ones(Int64,particle_number).*j,id= collect(1:particle_number),
                x = [i[1] for i in particle_list_alltime[j]],y = [i[2] for i in particle_list_alltime[j]]))
    end
    return Data
end

In [ ]:
length(particle_all)

In [ ]:
data = convert_to_DF(particle_all)

As the simulation data is now in form of a `DataFrame`, many of the `DataFrame` specific tools from the data_analysis notebook can be used. But first things first, let's save our data with the `CSV` package, just to be sure. 

In [ ]:
CSV.write("$(gridsize)_positions.csv",data)

It is good practice to save a settings file which includes all the parameters of the simulation together with your raw data. The easiest way to do that is a `dict`. 

In [ ]:
function save_settings(Gridsize,particle_number,Timesteps,Starting_config)
    settings = Dict("gridsize" => Gridsize, "particle_number" => particle_number, 
        "timesteps" =>Timesteps, "starting_config" => Starting_config)
    open("settings.txt","w") do file
        print(file,settings)
    end 
end

 ### 6. Putting everything together
 Because all the simulation steps are encapsulated in functions, it is possible to execute a whole simulation in one particle. This time with the "square" starting config. 

In [ ]:
gridsize = 200                               
particle_number = 1000                         
timesteps = 200                           
starting_config = "square"

grid, particle_list = create_objects(gridsize, particle_number)
grid, particle_list, particle_number = populate_sys!(grid, particle_list, starting_config,gridsize, particle_number)
particle_all_2 = update_sys(grid,particle_list,gridsize,timesteps)
data2 = convert_to_DF(particle_all_2)
CSV.write("$(gridsize)_positions.csv",data2)
save_settings(gridsize,particle_number,timesteps,starting_config)
fig3,ax3, scatty3 =plot_sim(particle_all_2[end],gridsize)
display(fig3)

### 10.7 Outlook: Let's make a beautiful animation :=)
You do not need to understand everything in this section, as concepts are used that have not been introduced yet. This should serve the purpose of showing you what is possible in Julia and also give you a very nice looking finish of this course. So sit back and enjoy ;).  
If you are interested in understanding what is going on here: Watch this [video](https://www.youtube.com/watch?v=L-gyDvhjzGQ ) of the brilliant Julia developer George Datseris, where he explains all the concepts needed to understand the code below. 

#### Short summary: 
Two functions are created here, one for a 2D animation and one for a 3D animation. Both need a grouped dataframe as argument together with the gridsize. Both functions return a figure object and a variable called t which represents the timesteps and is an object of type `Observable`. This `Observable` has the special ability, that all objects that depend on it (in this case x,y,r,titel and the figure) are automatically updated when it changes. Therefore, the `run_animation` function for both 2D and 3D is the same and just slowly increases the timesteps t. The plot then automatically updates :).

Note: Compared to other plotting packages like `matplotlib` in Python or `ggplot` in R, this is a very different approach to create animations or videos. The big advantage of the `Observable` based implementation is that you do not have to define a new plot object for every frame, you just need to specify what has changed between frames. Together with the gpu accelaration, this allows a much better performance.  

In [ ]:
function animation_2D(Data_gr,Gridsize) 
    t = Observable(1)
    x = @lift(Data_gr[$t].:x)                                          # @lift macro is needed when creating Observables which depend on other Observables
    y = @lift(Data_gr[$t].:y) 
    r = @lift((($x.-Gridsize/2).^2+($y.-Gridsize/2).^2).^(1/2))
    titel = @lift(string($t))

    set_theme!(theme_black())

    fig6 = Figure(resolution = (1000,1000))
    ax6 =Axis(fig6[1,1],title = @lift("timestep : $(round(Int64,$t))"),titlesize = 35)
    xlims!(ax6,0,Gridsize)
    ylims!(ax6,0,Gridsize)
    scatty6 = scatter!(ax6,x,y,color = r,colormap = :dense,label = "particle",marker = :circle,markersize = 15)

    #hidedecorations!(ax6)
    axislegend(ax6)
    display(fig6)
    return t,fig6
end

In [ ]:
function animation_3D(Data_gr,Gridsize) 
    
    # make sure that camera flight is not too fast
    if length(Data_gr) >= 1000
        len = length(Data_gr)
    else
        len = 1000
    end
    t = Observable(1)
    
    x = @lift(Data_gr[$t].:x)
    y = @lift(Data_gr[$t].:y)
    r = @lift((($x.-Gridsize/2).^2+($y.-Gridsize/2).^2).^(1/2))
    titel = @lift(string($t))
   
    # define the camera angle
    elevations = range(start = -2π,stop = 2π, length = len)
    azimuths = range(start = 0,stop = 2π, length = len)
    z = zeros(length(Data_gr[1].:x))
    set_theme!(theme_black())

    fig7 = Figure(resolution = (2000,2000))
    ax7 =Axis3(fig7[1,1],
        title = @lift("timestep : $(round(Int64,$t))"),
        elevation =@lift(elevations[$t])  , azimuth = @lift(azimuths[$t]),
        viewmode = :fit,
        titlesize = 35,protrusions = (0, 0, 0, 40)
        )   

    xlims!(ax7,0,Gridsize)
    ylims!(ax7,0,Gridsize)
    zlims!(-Gridsize/2,Gridsize/2)

    scatty7 = scatty = meshscatter!(ax7,x,y,z,markersize = 2.5,color = r,
        colormap = :diverging_gkr_60_10_c40_n256
        ,label = "particle")
    #hidedecorations!(ax7)
    display(fig7)
    return t,fig7
end

Additionally, a function for running the animation and one for running and saving the animation into a video, have to be defined. As all the plotting features have already been specified in the `animation_2D()` respectively `animation_3D()`, these functions only increment the `Observable` `t` by 1 in every frame. 

In [ ]:
function run_animation(T,Timesteps)
    @async for i in 2:1:Timesteps
        T[] = T[]+1
        sleep(0.05)
    end
    T[] = 1
end

In [ ]:
function save_animation(T,Fig,Data_gr)
    record(Fig, "beautiful2.mp4",1:length(Data_gr); framerate = 30) do i
    T[] = i
    end
end

Comment: You might be thinking that animating a 2D system in 3D is quite stupid and unneccessary. And you are right. From a scientifc perspective, that does not make much sense. But in this case, it serves other educational purposes:  

1. 2D and 3D plots/animations are quite similar (only add z-coordinate and define camera)
2. They can be updated, handled and saved with the same functions  
3. 3D is beautiful ;) 

Lets create some data to run an animation. 

In [ ]:
data_gr= groupby(data2,:timestep);

In [ ]:
t,fig_2d = animation_2D(data_gr,gridsize);

Run the 2D animation:

In [ ]:
run_animation(t,timesteps)

Create a 3D animation: 

In [ ]:
t3,fig_3d = animation_3D(data_gr,gridsize);

Run the 2D animation:

In [ ]:
run_animation(t3,timesteps)

##  Short  Summary
- An agent-based simulation is defined by a set of parameters
- In this case, the agents are representing particles and the 2D grid is representing a discretized 2D space 
- The model logical is encapsulated in functions and the model progreses in discret time steps 
- `GLMakie` and its `Observable` based approach are used to visualize the system in 2D and 3D 

## Exercises

### <p style='color: green'>easy</p>

1. Make some simulations for very small and very large (gridsize, particle_number) systems and admire the beautiful animations.
Hint: Make sure, that the number of particles fit onto the grid. 

2. The color scheme used in the 3D animation is not very friendly to people with a red-green deficiency. Change that to a more friendly color scheme. If you do not remember where to find a list of the available color schemes, take a second look at the plotting chapter of this course. 

3. Use the `save_animation()` function to save your favorite simulation as a video.  

### <p style='color: orange'>medium</p>

4. Define a new parameter `name` which should be part of the name of all created files of the simulation (csv, settingsfile, video). Change all involved functions accordingly. 

5. Normally, you would not put such long function definitions into a jupyter notebook and then use them in the same jupyter notebook below. Typically, you would put them into an external file and import them at the beginning. Take a look at how `include()` [works](https://docs.julialang.org/en/v1/manual/code-loading/). Create a `test.jl` file with an example function written by you and include it into a jupyter notebook and execute it there.   

6. Put all the functions defined here into a `random_walk.jl` file. Include the file into a new juypter notebook called `executor.ipynb` and run the simulation and animation there. 


7. Change the movement of the particles by changing the `update_sys()` function to `update_sys_bias()` in `random_walk.jl`. Remove one of the possible movement vectors on the grid to give them a bias in a certain direction. How does the result change for long time spans? Save a video of your changed simulation. 

Hint: If you are changing a `.jl` you have to include it again, so that the changes are active in the Julia session. 

### <p style='color: red'>hard</p>


8. The starting configuration in a square is not very realistic, add another option in the `populate_sys!` that puts the particles into a circle. 

### <p style='color: red'>Very hard - Only do when you want to punish yourself</p>

9. A typical way to quantify a random walk is the mean squared displacement [(MSD)](https://en.wikipedia.org/wiki/Mean_squared_displacement). Make a simulation run with the following parameters: 
```julia
gridsize = 1000                               
particle_number = 1000                         
timesteps = 5000                           
starting_config = "random"
name = "msd"
```
Use the created dataframe to calculate the MSD. Plot the MSD against time in a line plot and save it. What behaviour do you see and what is expected from a [random walk](https://en.wikipedia.org/wiki/Anomalous_diffusion)? 